# Flagging Loans Project


## Description:

The report will be made on data that contains information on repeated loans from current clients.

Using the available dataset, I will build a model that will predict the value of the target variable `bad_flag` and will argue for the choice of model chosen and show the expected default rate at different approval levels with detailed explanation.

### Let's import the necessary libraries

In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")